In [22]:
import pandas as pd
import numpy as np
import json 
import copy

dat = [] 
for line in open('review_train.json', 'r'): 
    dat.append(json.loads(line))

In [10]:
dat[0]

{'business_id': 31292,
 'stars': 1.0,
 'text': 'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.',
 'date': '2013-05-07 04:34:36'}

In [20]:
len(dat)


5364626

In [23]:
reviewsdf=pd.DataFrame.from_dict(dat)

In [25]:
reviewsdf

,business_id,date,stars,text
0,31292,2013-05-07 04:34:36,1.0,Total bill for this horrible service? Over $8G...
1,35344,2017-01-14 21:30:33,5.0,I *adore* Travis at the Hard Rock's new Kelly ...
2,152538,2016-11-09 20:09:03,5.0,I have to say that this office really has it t...
3,71871,2018-01-09 20:56:38,5.0,Went in for a lunch. Steak sandwich was delici...
4,64913,2018-01-30 23:07:38,1.0,Today was my second out of three sessions I ha...
5,112310,2013-01-20 13:25:59,4.0,I'll be the first to admit that I was not exci...
6,176386,2016-05-07 01:21:02,3.0,Tracy dessert had a big name in Hong Kong and ...
7,139555,2010-10-05 19:12:35,1.0,This place has gone down hill. Clearly they h...
8,50173,2012-02-29 21:52:43,3.0,It's a giant Best Buy with 66 registers. I do...
9,13172,2011-11-30 02:11:15,4.0,"Like walking back in time, every Saturday morn..."


In [26]:
reviewsdf.describe

<bound method NDFrame.describe of          business_id                 date  stars  \
0              31292  2013-05-07 04:34:36    1.0   
1              35344  2017-01-14 21:30:33    5.0   
2             152538  2016-11-09 20:09:03    5.0   
3              71871  2018-01-09 20:56:38    5.0   
4              64913  2018-01-30 23:07:38    1.0   
5             112310  2013-01-20 13:25:59    4.0   
6             176386  2016-05-07 01:21:02    3.0   
7             139555  2010-10-05 19:12:35    1.0   
8              50173  2012-02-29 21:52:43    3.0   
9              13172  2011-11-30 02:11:15    4.0   
10             40775  2017-12-15 23:27:08    1.0   
11            116739  2016-05-07 01:36:53    4.0   
12             12722  2018-04-27 20:25:26    4.0   
13            185188  2012-07-16 00:37:14    1.0   
14             10013  2017-04-07 21:27:49    5.0   
15             14110  2015-01-03 22:47:34    4.0   
16             33304  2017-05-26 01:23:19    5.0   
17             97870  2014-06-

In [28]:
highRW=reviewsdf.loc[reviews['stars']>=4.0]

In [30]:
highRW.shape #3532298 out of 5364626

(3532298, 4)

In [ ]:
reviewslist=copy.deepcopy(dat)
import csv
keys = reviewslist[0].keys()
f = open("reviewsList.csv", "w")
writer = csv.DictWriter(
    f, fieldnames=keys)
writer.writeheader()
writer.writerows(
   reviewslist)
f.close()
reviewslist[0]

In [ ]:
reviewsTEXT=copy.deepcopy(reviewslist)
for ind in range(len(reviewslist)):
    texts=''
    texts = reviewslist[ind]['text']
    reviewsTEXT[ind]['text']=texts.split(' ')
    
reviewsTEXT[0]

#there are numbers, $, ? should be also remove. texts.split(' ') is not enough

In [ ]:
import re


REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\$)|(\*)")
REPLACE_WITH_SPACE = re.compile("(\n)|(\-)|(\/)|(\d)")

def preprocess_reviews(reviews):
    reviews = REPLACE_NO_SPACE.sub("", reviews)
    reviews = REPLACE_WITH_SPACE.sub(" ", reviews)
    return reviews

reviewsTEXT_clean = copy.deepcopy(reviewslist)

for ind in range(1):#len(reviewslist)
    texts=''
    texts = reviewslist[ind]['text']
    print(texts)
    text_clean=preprocess_reviews(reviewslist[ind]['text'])
    print(text_clean)
    reviewsTEXT_clean[ind]['text']=text_clean.split()
    reviewsTEXT_clean[ind]['freq']=nltk.FreqDist(reviewsTEXT_clean[ind]['text'])
    
reviewsTEXT_clean[0]

In [ ]:
#exclude words from the texts
import nltk
#nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

for ind in range(len(reviewsTEXT_clean)):
    texts=''
    texts = reviewsTEXT_clean[ind]['text']
    textsLOW=[x.lower() for x in texts]
    filtered_text = [w for w in textsLOW if not w in stop_words] 
    reviewsTEXT_clean[ind]['text']=filtered_text

In [ ]:
AllFreq = reviewsTEXT_clean[0]['freq']
for ind in range(len(reviewsTEXT_clean)-1):
    freqs = reviewsTEXT_clean[ind+1]['freq']
    AllFreq.update(freqs)
    
sorted(AllFreq.items(),key=lambda item:item[1],reverse=True)